<a href="https://colab.research.google.com/github/alexlautw9527/for_notebook/blob/main/Fund2vec_ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd 
import numpy as np

In [4]:
fund_code_df = pd.read_csv('/content/drive/MyDrive/05_OTHER_DATA/FUND_CODE_DATA.csv', encoding='CP950')
fund_code_df = fund_code_df[['產品代碼', '基金名稱', 'ISIN Code']]

In [5]:
fund_df = pd.read_csv('/content/drive/MyDrive/05_OTHER_DATA/基金產品列表.csv', encoding='CP950')
fund_df = fund_df[fund_df['RISKLEVEL']!=0][['FUND_CD','CNAME','FUNDCRYID', 'RISKLEVEL','DIVIDENDTYPE2','ASSETCAT_CODE','SECTOR_NAME']]
fund_df['SECTOR_NAME'] = fund_df['SECTOR_NAME'].fillna('NA')
fund_df = fund_df[~fund_df['ASSETCAT_CODE'].isna()]
fund_df = fund_df.astype(str)

In [6]:
org_fund_corpus = fund_df[['CNAME','FUNDCRYID', 'RISKLEVEL','DIVIDENDTYPE2','ASSETCAT_CODE','SECTOR_NAME']].astype(str).values.tolist()

In [7]:
fund_graph = pd.get_dummies(fund_df, columns=['FUNDCRYID', 'RISKLEVEL','DIVIDENDTYPE2','ASSETCAT_CODE','SECTOR_NAME'])
fund_graph = fund_graph.set_index('CNAME').stack()
fund_graph = fund_graph[fund_graph==1]
edges = fund_graph.index.tolist()

In [8]:
import networkx as nx
G = nx.Graph(edges)

In [9]:
!pip install node2vec
!pip install glove-python-binary

In [ ]:
import pprint
import gensim
from glove import Glove
from glove import Corpus


sentense = org_fund_corpus
corpus_model = Corpus()
corpus_model.fit(sentense, window=10,)

print('Dict size: %s' % len(corpus_model.dictionary))
print('Collocations: %s' % corpus_model.matrix.nnz)


glove = Glove(no_components=50, learning_rate=0.05,)
glove.fit(corpus_model.matrix, epochs=20,
          no_threads=1, verbose=True)
glove.add_dictionary(corpus_model.dictionary)


glove.save('glove.model')
glove = Glove.load('glove.model')


corpus_model.save('corpus.model')
corpus_model = Corpus.load('corpus.model')


In [32]:
np.savetxt("fund_embedd_glove.tsv", glove.word_vectors[[glove.dictionary[i] for i in fund_df['CNAME'].tolist()]], delimiter="\t")

In [12]:
import networkx as nx
from node2vec import Node2Vec

# Precompute probabilities and generate walks - **ON WINDOWS ONLY WORKS WITH workers=1**
node2vec = Node2Vec(G, dimensions=64, walk_length=80, num_walks=10, workers=8)  # Use temp_folder for big graphs

Computing transition probabilities:   0%|          | 0/2595 [00:00<?, ?it/s]

In [16]:
import pandas as pd
from gensim.models.word2vec import Word2Vec

In [ ]:
model = Word2Vec(corpus, size=50, iter=50,window=5, sg=1,negative=20,min_count=1)


In [21]:
n2v_model = Word2Vec(node2vec.walks, size=100, iter=10, window=3, sg=1, negative=20,min_count=1,workers=16)

In [ ]:
def most_similar(w2v_model, words, topn=20):
    similar_df = pd.DataFrame()
    for word in words:
        try:
            similar_words = pd.DataFrame(w2v_model.wv.most_similar(word, topn=topn), columns=[word, 'cos'])
            similar_df = pd.concat([similar_df, similar_words], axis=1)
        except:
            print(word, "not found in Word2Vec model!")
    return similar_df

In [22]:
np.savetxt("fund_embedd_n2v.tsv", n2v_model.wv[fund_df['CNAME'].tolist()], delimiter="\t")


In [ ]:
fund_df.to_csv('fund_embedd_metadata.tsv', sep = '\t',index=False)

In [ ]:
fund_df.shape

(2650, 6)

In [ ]:
from google.colab import files
files.download('/content/fund_embedd_metadata.tsv') 
files.download('/content/fund_embedd.tsv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
! wget https://www.rahimikia.com/FinText/FinText_FastText_Skip-gram.zip

--2021-10-26 15:11:10--  https://www.rahimikia.com/FinText/FinText_FastText_Skip-gram.zip
Resolving www.rahimikia.com (www.rahimikia.com)... 69.163.228.90
Connecting to www.rahimikia.com (www.rahimikia.com)|69.163.228.90|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11656495869 (11G) [application/zip]
Saving to: ‘FinText_FastText_Skip-gram.zip’

FinText_FastText_Sk 100%[===================>]  10.86G  13.1MB/s    in 8m 13s  

2021-10-26 15:19:24 (22.5 MB/s) - ‘FinText_FastText_Skip-gram.zip’ saved [11656495869/11656495869]



In [ ]:
!unzip /content/FinText_FastText_Skip-gram.zip

Archive:  /content/FinText_FastText_Skip-gram.zip
   creating: FinText_FastText_Skip-gram/
  inflating: FinText_FastText_Skip-gram/Word_Embedding_2000_2015  
  inflating: FinText_FastText_Skip-gram/README.txt  
  inflating: FinText_FastText_Skip-gram/Word_Embedding_2000_2015.trainables.vectors_ngrams_lockf.npy  
  inflating: FinText_FastText_Skip-gram/Word_Embedding_2000_2015.pkl  
  inflating: FinText_FastText_Skip-gram/Word_Embedding_2000_2015.wv.vectors_ngrams.npy  
  inflating: FinText_FastText_Skip-gram/Word_Embedding_2000_2015.wv.vectors.npy  
  inflating: FinText_FastText_Skip-gram/Word_Embedding_2000_2015.trainables.vectors_vocab_lockf.npy  
  inflating: FinText_FastText_Skip-gram/Word_Embedding_2000_2015.wv.vectors_vocab.npy  
  inflating: FinText_FastText_Skip-gram/Word_Embedding_2000_2015.trainables.syn1neg.npy  


In [ ]:
from gensim.models import Word2Vec
from gensim.models import FastText
import gensim.downloader as api 

In [ ]:
FinText_Word2Vec_skipgram = Word2Vec.load('/content/FinText_FastText_Skip-gram/Word_Embedding_2000_2015')

In [ ]:

prod=['fund',
 'etf',
 'stock',
 'preferred_stock',
 'auto_loan',
 'trust',
 'life_insurance',
 'insurance',
 'personal_loan',
 'mortgage',
 'loan',
 'treasury_bond',
 'foreign_exchange',
 'online_banking',
 'fintech',
 'blockchain',
 'bitcoin',
 'robo_advisor'
 ]

In [ ]:
product_embedding = FinText_Word2Vec_skipgram.wv[prod]

In [ ]:
product_embedding

array([[ 0.14604235,  0.1704098 ,  0.01929267, ..., -0.13392453,
        -0.3286863 ,  0.01605617],
       [ 0.40752319,  0.3141233 , -0.22489968, ..., -0.28437993,
         0.02246348,  0.17138132],
       [ 0.09119733,  0.13523722,  0.01482047, ..., -0.09712761,
        -0.23065072, -0.22710954],
       ...,
       [-0.00303637,  0.145203  , -0.41418603, ..., -0.11681274,
        -0.14459747, -0.02398938],
       [-0.07647095,  0.24916057, -0.12600106, ...,  0.3001527 ,
        -0.21294053, -0.04931631],
       [ 0.29539302, -0.00322265,  0.12641694, ...,  0.13217312,
        -0.4011273 , -0.07066225]], dtype=float32)

In [ ]:
product_embedding.shape

(18, 300)

In [ ]:
np.savetxt("product_embedd.tsv", product_embedding, delimiter="\t")


In [ ]:
pd.DataFrame({'product':prod}).to_csv('prod_embedd_metadata.tsv', sep = '\t',index=False,header=False)

In [ ]:
from google.colab import files
files.download('/content/prod_embedd_metadata.tsv') 
files.download('/content/product_embedd.tsv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>